<a href="https://colab.research.google.com/github/anjha1/Data-Science/blob/main/Python/180.%20Random%20Forest%20Practical%20Implementation%20With%20Pipeline/Random_Forest_Classifier_With_Pipeline_And_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random Forest Classifier With Pipeline And Hyperparameter Tuning**

In [49]:
import seaborn as sns
df=sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [50]:
df.isnull().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [51]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [52]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [53]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [54]:
df.time

,time
0,Dinner
1,Dinner
2,Dinner
3,Dinner
4,Dinner
...,...
239,Dinner
240,Dinner
241,Dinner
242,Dinner


In [55]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [56]:
df.time

,time
0,0
1,0
2,0
3,0
4,0
...,...
239,0
240,0
241,0
242,0


In [57]:
df.time.unique()

array([0, 1])

In [58]:
X=df.drop(labels=['time'],axis=1)
y=df['time']

In [59]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [60]:
y

,time
0,0
1,0
2,0
3,0
4,0
...,...
239,0
240,0
241,0
242,0


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

[link Pipeline](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html)


[link SimpleImputer](https://scikit-learn.org/1.5/modules/generated/sklearn.impute.SimpleImputer.html)



[link ColumnTransformer](https://scikit-learn.org/1.5/modules/generated/sklearn.compose.ColumnTransformer.html)

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer   #Handling Missing Value
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.preprocessing import OneHotEncoder #Catogerical to Nomerical
from sklearn.compose import ColumnTransformer

In [63]:
categorical_features=['sex','smoker','day']
numerical_features=['total_bill','tip','size']

In [64]:
#Numrical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), #Handling Missing Value
        ('scaler',StandardScaler())                   #Feature Scaling
    ]
)

#Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), #Missing Value
        ('onehotencoder',OneHotEncoder())             #Catogerical to Nomerical
    ]
)

In [65]:
preprocesser=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_features),
    ('cat_pipeline',cat_pipeline,categorical_features)
])

In [66]:
X_train=preprocesser.fit_transform(X_train)
X_test=preprocesser.transform(X_test)

In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [68]:
# Model Training Automation
# models={
#     'RandomForestClassifier':RandomForestClassifier(),
#     'LogisticRegression':LogisticRegression(),
#     'DecisionTreeClassifier':DecisionTreeClassifier(),
#     'GradientBoostingClassifier':GradientBoostingClassifier(),
#     'KNeighborsClassifier':KNeighborsClassifier(),
# }

In [69]:
models={
    'RandomForestClassifier':RandomForestClassifier(),
    'LogisticRegression':LogisticRegression(),
    'DecisionTreeClassifier':DecisionTreeClassifier()
}

In [70]:
from sklearn.metrics import accuracy_score

In [71]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)



        y_test_pred=model.predict(X_test)
        test_model_score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]]=test_model_score
    return report

In [72]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'RandomForestClassifier': 0.9591836734693877,
 'LogisticRegression': 1.0,
 'DecisionTreeClassifier': 0.9387755102040817}

In [73]:
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [74]:
params = {
    'max_depth': [3, 5, 10, None],
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy']
}

In [75]:
from sklearn.model_selection import RandomizedSearchCV

In [76]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [77]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.974 total time=   1.5s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.923 total time=   1.6s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.974 total time=   1.7s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.949 total time=   1.5s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.949 total time=   1.3s
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.974 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.923 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.923 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [78]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'gini'}

In [79]:
cv.predict(X_test)

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1])